<center><h1> Rucode 6.0 Car Color Detection</h1></center>

To reproduce result sucessfully is needed:
- Download provided by organizators zip archive with train data and rename it to `dataset.zip`
- Launch this notebook.

Required imports

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.image as img
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, losses, utils

Unzip dataset archive

In [ ]:
! rm -r sample_data &> /dev/null
! rm -r public_test sample_submission.csv train &> /dev/null
! unzip dataset.zip &> /dev/null
! ls train

Dataset separated to train data and test data needed to evaluate

In [ ]:
train_dir = 'train'
test_dir = 'public_test'

Take a look at some images and their props.

In [ ]:
print("Some shapes:", img.imread("train/Yellow/3567.jpg").shape, img.imread("train/Red/3011.jpg").shape)
print("Max value of some image:", img.imread("train/Yellow/3567.jpg").max())
print("dtype:", img.imread("train/Yellow/3567.jpg").dtype)

print("Some image:")
# plt.imshow(img.imread("train/Yellow/3567.jpg"));
# plt.imshow(img.imread("train/Yellow/3751.jpg"));
# plt.imshow(img.imread("train/Yellow/3633.jpg"));
plt.imshow(img.imread("train/Yellow/3498.jpg"));
# plt.imshow(img.imread("train/Red/3011.jpg"));
# plt.imshow(img.imread("train/Grey/1533.jpg"));

As far as we see shapes are different and we need to tramsform images to equal shapes.

We can use low-resolution shape because it's not required to work with hi-res to determinate color. Let it be equal to $(32, 32)$.

In [ ]:
overall_shape = (32, 32)

Let's import train dataset to in-memory representaion.

In [ ]:
labels = []

train_imgs = []
train_lbls = []

# iterate over all labels in train dataset
for i, label in enumerate(os.listdir(train_dir)):
  labels.append(label)
  dir = os.path.join(train_dir, label)

  # iterate over all images of label
  for filename in os.listdir(dir):
    filepath = os.path.join(dir, filename)
    image = Image.open(filepath)
    image = np.array(image.resize(overall_shape))
    train_imgs.append(image)
    train_lbls.append(i)
    
train_imgs = np.stack(train_imgs)
train_lbls = np.stack(train_lbls)

train_imgs.shape, train_lbls.shape

Let's import test dataset to in-memory representaion.

In [ ]:
test_imgs = []
sorted_files = sorted(os.listdir(test_dir), key=lambda x: int(x.split('.')[0]))

# iterate over sorted by name images of dataset
for filename in sorted_files:
  filepath = os.path.join(test_dir, filename)
  image = Image.open(filepath)
  image = np.array(image.resize(overall_shape))
  test_imgs.append(image)

test_imgs = np.stack(test_imgs)

test_imgs.shape

Apply data transforamtions to them:

* Normalise train data (min-max normalistion is common choise for imges)
* Make labels as categorical (with one-hot encoding) 

In [ ]:
# normalize images
train_imgs = train_imgs / 255.0
test_imgs = test_imgs / 255.0
print("min:", train_imgs.min(), "max:", train_imgs.max())

# encode labels
train_lbls = utils.to_categorical(train_lbls, len(labels))
train_lbls.shape

Split training dataset on training data and validation data in proportion $8 : 2$

In [ ]:
training_data_rate = 0.8
training_data_size = int(training_data_rate * len(train_imgs))

permut = np.random.RandomState(seed=42).permutation(len(train_imgs))

valid_imgs = train_imgs[permut[training_data_size:]]
valid_lbls = train_lbls[permut[training_data_size:]]

train_imgs = train_imgs[permut[:training_data_size]]
train_lbls = train_lbls[permut[:training_data_size]]

len(train_imgs), len(valid_imgs)

Let's create some simple convolutional network for classification

In [ ]:
model = models.Sequential()

# module of convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=train_imgs.shape[1:]))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# classification module with softmax at the end
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(labels), activation='softmax'))

# summary
model.summary()

Compile model and choose hyperparameters

In [ ]:
model.compile(loss=losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

epoches = 15 # to change

Train model

In [ ]:
history = model.fit(train_imgs, train_lbls, epochs=epoches, validation_data=(valid_imgs, valid_lbls))

Training process visualisation

In [ ]:
plt.figure(figsize=(14,8))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

model.evaluate(valid_imgs,  valid_lbls, verbose=2)
None

Make prediction

In [ ]:
test_logits = model.predict(test_imgs)
test_lbls = np.argmax(test_logits, axis=1)
test_ans = np.array(labels)[test_lbls]
test_ans[:5]

Check answer format

In [ ]:
sample = pd.read_csv('sample_submission.csv', index_col=False)
sample.head()

Serialize answer

In [ ]:
ans_df = pd.DataFrame(test_ans)
ans_df.to_csv('submit.csv', index=False, header=False)
ans_df.head()

-------

**Made by Bronnikov Maksim**